In [1]:
%matplotlib inline
# Set below to 0 or 1 for GPU
%env CUDA_VISIBLE_DEVICES=
from __future__ import absolute_import, division, print_function
import json
import os
if os.path.isfile("config.json"):
    with open("config.json", "r") as f:
        config = json.load(f)
else:
    print("Please run setup.py in this directory before running any .ipynb's.")

import matplotlib.pyplot as plt
import rosbag
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
import time
import shutil
import pickle
from collections import OrderedDict, defaultdict

from utils.bags import *
from utils.learning import *
from traffic_weaving_models import *

config

env: CUDA_VISIBLE_DEVICES=


{u'catkin_ws_dir': u'/home/schmrlng/catkin_ws',
 u'data_dir': u'/data/trafficweaving',
 u'feature_extraction_dict': u'slim',
 u'models_dir': u'/data/trafficweaving/models'}

In [2]:
model_type = TrafficWeavingCVAE
cvd = %env CUDA_VISIBLE_DEVICES
model_dir = os.path.join(config["models_dir"], ("GPU" + cvd if cvd else "CPU") + "_checkpoints")
# shutil.rmtree(model_dir, ignore_errors=True)

sc = tf.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=False)
rc = tf.estimator.RunConfig().replace(session_config=sc, model_dir=model_dir,
                                      save_summary_steps=100, save_checkpoints_steps=5000,
                                      keep_checkpoint_max=5000, tf_random_seed=None)
rc.environment = None    # required due to a bug in tf.contrib.learn.Experiment.train_and_evaluate
model_dir

u'/data/trafficweaving/models/CPU_checkpoints'

In [3]:
# Edit model hyperparameters if desired, e.g.:
# hps.N = 2
# hps.K = 5
hps.values()

{'GMM_components': 16,
 'K': 5,
 'MLP_dropout_keep_prob': 0.9,
 'MVG_latent_dim': 32,
 'N': 2,
 'alpha': 1,
 'batch_size': 32,
 'dec_GMM_proj_MLP_dims': None,
 'dec_rnn_dim': [128],
 'dec_sample_model_prob_crossover': 20000,
 'dec_sample_model_prob_divisor': 6,
 'dec_sample_model_prob_final': 0.0,
 'dec_sample_model_prob_start': 0.0,
 'enc_rnn_dim_future': [32],
 'enc_rnn_dim_history': [32],
 'fuzz_factor': 0.05,
 'grad_clip': 1.0,
 'k': 3,
 'k_eval': 50,
 'kl_crossover': 8000,
 'kl_decay_rate': 0.99995,
 'kl_exact': True,
 'kl_min': 0.07,
 'kl_sigmoid_divisor': 6,
 'kl_weight': 1.0,
 'kl_weight_start': 0.0001,
 'latent_type': 'discrete',
 'learning_decay_rate': 0.9999,
 'learning_rate': 0.001,
 'log_p_yt_xz_max': 50,
 'log_sigma_max': 10,
 'log_sigma_min': -10,
 'min_learning_rate': 1e-05,
 'minimum_history_length': 5,
 'optimizer': tensorflow.python.training.adam.AdamOptimizer,
 'optimizer_kwargs': {},
 'p_z_x_MLP_dims': [16],
 'prediction_horizon': 15,
 'q_z_xy_MLP_dims': None,
 'rn

In [4]:
extract_dict = feature_extraction_dicts[config["feature_extraction_dict"]]
array_name = "trajectories_" + config["feature_extraction_dict"]
base_filename = os.path.join(config["data_dir"], array_name)

all_bags = pd.read_pickle(base_filename + ".pkl")
train_bags, eval_bags, test_bags = split_bag_df(all_bags, train_eval_test_split = [.8, .2, 0.], seed=123)

train_data_dict = prepare_data(train_bags, extract_dict=extract_dict)
save_as_npz_and_hdf5(base_filename + "_train", train_data_dict["input_dict"])
hps.add_hparam("cars_standardization", {"mean": train_data_dict["cars_mean"],
                                        "std": train_data_dict["cars_std"]})
hps.add_hparam("extras_standardization", {"mean": train_data_dict["extras_mean"],
                                          "std": train_data_dict["extras_std"]})
hps.add_hparam("label_standardization", {"mean": train_data_dict["labels_mean"],
                                         "std": train_data_dict["labels_std"]})

hps.add_hparam("pred_indices", train_data_dict["pred_indices"])
eval_data_dict = prepare_data(eval_bags, extract_dict=extract_dict)
save_as_npz_and_hdf5(base_filename + "_eval", eval_data_dict["input_dict"])

train_input_function = tf.estimator.inputs.numpy_input_fn(train_data_dict["input_dict"],
                                                          y = train_data_dict["labels"],
                                                          batch_size = hps.batch_size,
                                                          num_epochs = None,
                                                          shuffle = True)

eval_input_function = tf.estimator.inputs.numpy_input_fn(eval_data_dict["input_dict"],
                                                         y = eval_data_dict["labels"],
                                                         batch_size = 4,
                                                         num_epochs = 1,
                                                         shuffle = False)

In [5]:
laneswap_model = model_type()
nn = tf.estimator.Estimator(laneswap_model.model_fn, params=hps, config=rc, model_dir=model_dir)
experiment = tf.contrib.learn.Experiment(nn, train_input_function, eval_input_function,
                                         train_steps=50000, min_eval_frequency=1, eval_steps=None)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': device_count {
  key: "GPU"
  value: 1
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5000, '_tf_random_seed': None, 'environment': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': 5000, '_model_dir': u'/data/trafficweaving/models/CPU_checkpoints', '_save_summary_steps': 100}


In [6]:
experiment.train_and_evaluate()

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /data/trafficweaving/models/GPU1_checkpoints/model.ckpt.
INFO:tensorflow:Starting evaluation at 2017-10-13-08:27:38
INFO:tensorflow:Restoring parameters from /data/trafficweaving/models/GPU1_checkpoints/model.ckpt-1
INFO:tensorflow:Finished evaluation at 2017-10-13-08:27:46
INFO:tensorflow:Saving dict for global step 1: NLL_exact = 43.0704, NLL_p = 43.0737, NLL_q (IS) = 43.0869, global_step = 1, loss = 43.0869
INFO:tensorflow:Validation (step 1): loss = 43.0869, NLL_p = 43.0737, NLL_exact = 43.0704, NLL_q (IS) = 43.0869, global_step = 1
INFO:tensorflow:loss = 39.3238, step = 1
INFO:tensorflow:kl_weight = 0.00257238, lr = 0.001, temp = 2.0
INFO:tensorflow:global_step/sec: 5.06749
INFO:tensorflow:loss = 11.1289, step = 101 (10.231 sec)
INFO:tensorflow:kl_weight = 0.00276442, lr = 0.000990147, temp = 1.98011 (10.2

INFO:tensorflow:loss = -19.5202, step = 4001 (9.703 sec)
INFO:tensorflow:kl_weight = 0.0475211, lr = 0.00067356, temp = 1.34085 (9.703 sec)
INFO:tensorflow:global_step/sec: 10.3309
INFO:tensorflow:loss = -19.9021, step = 4101 (9.680 sec)
INFO:tensorflow:kl_weight = 0.0510264, lr = 0.000666956, temp = 1.32752 (9.680 sec)
INFO:tensorflow:global_step/sec: 10.2922
INFO:tensorflow:loss = -19.9697, step = 4201 (9.716 sec)
INFO:tensorflow:kl_weight = 0.0547758, lr = 0.000660417, temp = 1.31432 (9.716 sec)
INFO:tensorflow:global_step/sec: 10.2536
INFO:tensorflow:loss = -18.7547, step = 4301 (9.753 sec)
INFO:tensorflow:kl_weight = 0.0587843, lr = 0.000653944, temp = 1.30125 (9.753 sec)
INFO:tensorflow:global_step/sec: 10.289
INFO:tensorflow:loss = -19.802, step = 4401 (9.719 sec)
INFO:tensorflow:kl_weight = 0.0630671, lr = 0.000647535, temp = 1.28831 (9.719 sec)
INFO:tensorflow:global_step/sec: 10.2785
INFO:tensorflow:loss = -20.1633, step = 4501 (9.729 sec)
INFO:tensorflow:kl_weight = 0.067639

INFO:tensorflow:loss = -21.2545, step = 8201 (9.766 sec)
INFO:tensorflow:kl_weight = 0.537476, lr = 0.00044595, temp = 0.881267 (9.766 sec)
INFO:tensorflow:global_step/sec: 10.2756
INFO:tensorflow:loss = -18.5694, step = 8301 (9.731 sec)
INFO:tensorflow:kl_weight = 0.556058, lr = 0.000441611, temp = 0.872506 (9.732 sec)
INFO:tensorflow:global_step/sec: 10.3114
INFO:tensorflow:loss = -21.0066, step = 8401 (9.698 sec)
INFO:tensorflow:kl_weight = 0.574485, lr = 0.000437316, temp = 0.863833 (9.697 sec)
INFO:tensorflow:global_step/sec: 10.1548
INFO:tensorflow:loss = -18.775, step = 8501 (9.849 sec)
INFO:tensorflow:kl_weight = 0.592707, lr = 0.000433063, temp = 0.855246 (9.849 sec)
INFO:tensorflow:global_step/sec: 10.2926
INFO:tensorflow:loss = -18.6899, step = 8601 (9.716 sec)
INFO:tensorflow:kl_weight = 0.610678, lr = 0.000428853, temp = 0.846744 (9.716 sec)
INFO:tensorflow:global_step/sec: 10.2085
INFO:tensorflow:loss = -19.9572, step = 8701 (9.795 sec)
INFO:tensorflow:kl_weight = 0.62835

INFO:tensorflow:global_step/sec: 10.2416
INFO:tensorflow:loss = -18.911, step = 12401 (9.764 sec)
INFO:tensorflow:kl_weight = 0.964432, lr = 0.000296414, temp = 0.579324 (9.765 sec)
INFO:tensorflow:global_step/sec: 10.138
INFO:tensorflow:loss = -17.3913, step = 12501 (9.864 sec)
INFO:tensorflow:kl_weight = 0.966917, lr = 0.000293563, temp = 0.573568 (9.864 sec)
INFO:tensorflow:global_step/sec: 10.2628
INFO:tensorflow:loss = -20.752, step = 12601 (9.744 sec)
INFO:tensorflow:kl_weight = 0.969234, lr = 0.000290741, temp = 0.56787 (9.744 sec)
INFO:tensorflow:global_step/sec: 10.2785
INFO:tensorflow:loss = -20.9638, step = 12701 (9.729 sec)
INFO:tensorflow:kl_weight = 0.971394, lr = 0.000287947, temp = 0.562228 (9.729 sec)
INFO:tensorflow:global_step/sec: 10.2877
INFO:tensorflow:loss = -20.7234, step = 12801 (9.720 sec)
INFO:tensorflow:kl_weight = 0.973406, lr = 0.000285181, temp = 0.556643 (9.720 sec)
INFO:tensorflow:global_step/sec: 10.3312
INFO:tensorflow:loss = -21.785, step = 12901 (9.

INFO:tensorflow:kl_weight = 0.998299, lr = 0.000200062, temp = 0.384771 (9.738 sec)
INFO:tensorflow:global_step/sec: 10.267
INFO:tensorflow:loss = -19.7244, step = 16601 (9.740 sec)
INFO:tensorflow:kl_weight = 0.998422, lr = 0.00019817, temp = 0.380952 (9.740 sec)
INFO:tensorflow:global_step/sec: 10.3698
INFO:tensorflow:loss = -22.3064, step = 16701 (9.644 sec)
INFO:tensorflow:kl_weight = 0.998536, lr = 0.000196297, temp = 0.37717 (9.644 sec)
INFO:tensorflow:global_step/sec: 10.2423
INFO:tensorflow:loss = -21.5866, step = 16801 (9.763 sec)
INFO:tensorflow:kl_weight = 0.998642, lr = 0.000194443, temp = 0.373426 (9.763 sec)
INFO:tensorflow:global_step/sec: 10.2929
INFO:tensorflow:loss = -17.6243, step = 16901 (9.715 sec)
INFO:tensorflow:kl_weight = 0.99874, lr = 0.000192608, temp = 0.36972 (9.714 sec)
INFO:tensorflow:global_step/sec: 10.3
INFO:tensorflow:loss = -19.9128, step = 17001 (9.709 sec)
INFO:tensorflow:kl_weight = 0.998831, lr = 0.00019079, temp = 0.36605 (9.709 sec)
INFO:tensor

...

INFO:tensorflow:kl_weight = 1.0, lr = 4.4021e-05, temp = 0.0696949 (9.788 sec)
INFO:tensorflow:global_step/sec: 10.1592
INFO:tensorflow:loss = -21.1574, step = 33801 (9.843 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 4.36824e-05, temp = 0.0690112 (9.843 sec)
INFO:tensorflow:global_step/sec: 10.2464
INFO:tensorflow:loss = -18.7407, step = 33901 (9.759 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 4.33472e-05, temp = 0.0683343 (9.759 sec)
INFO:tensorflow:global_step/sec: 10.3083
INFO:tensorflow:loss = -23.9377, step = 34001 (9.701 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 4.30153e-05, temp = 0.0676642 (9.702 sec)
INFO:tensorflow:global_step/sec: 10.1891
INFO:tensorflow:loss = -20.9409, step = 34101 (9.814 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 4.26867e-05, temp = 0.0670007 (9.814 sec)
INFO:tensorflow:global_step/sec: 10.2666
INFO:tensorflow:loss = -19.5643, step = 34201 (9.741 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 4.23614e-05, temp = 0.0663439 (9.741 sec)
INFO:tensorflow:global_ste

INFO:tensorflow:global_step/sec: 10.1736
INFO:tensorflow:loss = -20.1749, step = 38001 (9.829 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 3.21289e-05, temp = 0.0456825 (9.829 sec)
INFO:tensorflow:global_step/sec: 10.2776
INFO:tensorflow:loss = -19.2404, step = 38101 (9.730 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 3.19087e-05, temp = 0.0452378 (9.730 sec)
INFO:tensorflow:global_step/sec: 10.2075
INFO:tensorflow:loss = -18.6933, step = 38201 (9.797 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 3.16906e-05, temp = 0.0447975 (9.797 sec)
INFO:tensorflow:global_step/sec: 10.1551
INFO:tensorflow:loss = -20.2038, step = 38301 (9.847 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 3.14747e-05, temp = 0.0443616 (9.847 sec)
INFO:tensorflow:global_step/sec: 10.1395
INFO:tensorflow:loss = -24.5397, step = 38401 (9.863 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 3.1261e-05, temp = 0.0439301 (9.863 sec)
INFO:tensorflow:global_step/sec: 10.2603
INFO:tensorflow:loss = -21.4077, step = 38501 (9.746 sec)
INFO:te

INFO:tensorflow:kl_weight = 1.0, lr = 2.45384e-05, temp = 0.0303558 (9.809 sec)
INFO:tensorflow:global_step/sec: 10.1393
INFO:tensorflow:loss = -20.1294, step = 42301 (9.862 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 2.43937e-05, temp = 0.0300637 (9.863 sec)
INFO:tensorflow:global_step/sec: 10.0954
INFO:tensorflow:loss = -18.5226, step = 42401 (9.906 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 2.42505e-05, temp = 0.0297744 (9.906 sec)
INFO:tensorflow:global_step/sec: 10.0278
INFO:tensorflow:loss = -20.3525, step = 42501 (9.972 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 2.41086e-05, temp = 0.029488 (9.972 sec)
INFO:tensorflow:global_step/sec: 10.1034
INFO:tensorflow:loss = -20.7751, step = 42601 (9.898 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 2.39682e-05, temp = 0.0292045 (9.898 sec)
INFO:tensorflow:global_step/sec: 10.2127
INFO:tensorflow:loss = -21.107, step = 42701 (9.791 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 2.38292e-05, temp = 0.0289238 (9.791 sec)
INFO:tensorflow:global_step

INFO:tensorflow:global_step/sec: 10.1599
INFO:tensorflow:loss = -19.0789, step = 46501 (9.843 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 1.94565e-05, temp = 0.0200945 (9.843 sec)
INFO:tensorflow:global_step/sec: 10.1027
INFO:tensorflow:loss = -22.5587, step = 46601 (9.898 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 1.93624e-05, temp = 0.0199044 (9.898 sec)
INFO:tensorflow:global_step/sec: 10.1808
INFO:tensorflow:loss = -22.6834, step = 46701 (9.823 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 1.92692e-05, temp = 0.0197163 (9.823 sec)
INFO:tensorflow:global_step/sec: 10.1575
INFO:tensorflow:loss = -20.8336, step = 46801 (9.845 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 1.91769e-05, temp = 0.01953 (9.845 sec)
INFO:tensorflow:global_step/sec: 10.1664
INFO:tensorflow:loss = -18.0127, step = 46901 (9.836 sec)
INFO:tensorflow:kl_weight = 1.0, lr = 1.90856e-05, temp = 0.0193456 (9.836 sec)
INFO:tensorflow:global_step/sec: 10.2062
INFO:tensorflow:loss = -20.0049, step = 47001 (9.798 sec)
INFO:ten

({'NLL_exact': -27.993811,
  'NLL_p': -27.798567,
  'NLL_q (IS)': -27.88678,
  'global_step': 50000,
  'loss': -27.88678},
 [])

In [7]:
state_dim = train_data_dict["input_dict"]["car1"].shape[2]
extras_dim = train_data_dict["input_dict"]["extras"].shape[2]
ph = hps.prediction_horizon

with tf.Graph().as_default():
    serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(
        {"car1": tf.placeholder(tf.float32, shape=[1, None, state_dim], name="car1"),
         "car2": tf.placeholder(tf.float32, shape=[1, None, state_dim], name="car2"),
         "extras": tf.placeholder(tf.float32, shape=[1, None, extras_dim], name="extras"),
         "traj_lengths": tf.placeholder(tf.int32, shape=[1], name="traj_lengths"),
         "car1_future_x": tf.placeholder(tf.float32, shape=[None, ph, state_dim/2], name="car1_future_x"),
         "car1_future_y": tf.placeholder(tf.float32, shape=[None, ph, state_dim/2], name="car1_future_y"),
         "sample_ct": tf.placeholder(tf.int32, shape=[1], name="sample_ct")})
    nn.export_savedmodel(config["models_dir"], serving_input_receiver_fn,
#                        checkpoint_path="/home/schmrlng/tf_training/laneswap_cvae/CPU/model.ckpt-11001"
                        )

INFO:tensorflow:Restoring parameters from /data/trafficweaving/models/GPU1_checkpoints/model.ckpt-50000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /data/trafficweaving/models/1507924360/saved_model.pb
